<a href="https://colab.research.google.com/github/jgonzet/7506-Org.-de-Datos/blob/master/exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Click para importar librerías

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Lectura y limpieza básica:

Se levanta el csv y declaran variables categoricas como tales y algunas fechas
se ponen en su formato. 
Se rellena territorio vacio Japon.
Se filtran regiones o territorios nan.
Se calcula para cada oportunidad su monto total. 
Se filtran las oportunidades que no son Won/Lost.
Se transforma Stage a columna del tipo booleana.

In [4]:
# Levantamos el csv declarando algunas variables categoricas:
datos = pd.read_csv("/content/Entrenamieto_ECI_2020.csv",dtype=
                   {#'Region': 'category',
                    #'Territory': 'category',
                    'Source': 'category',
                    'Billing_Country': 'category',
                    'Account_Name': 'category',
                    'Brand': 'category',
                    'ASP_Currency': 'category',
                    'ASP_(converted)_Currency': 'category',
                    'Source ': 'category',
                    #'Stage' : 'category'
                    })

#pd.set_option('display.max_columns', None) # Cuidado con esta opcion, usar solo cuando haga falta

# Convertimos las fechas:
datos['Account_Created_Date'] = pd.to_datetime(datos['Account_Created_Date'])
datos['Opportunity_Created_Date'] = pd.to_datetime(datos['Opportunity_Created_Date'])
datos['Planned_Delivery_Start_Date'] = pd.to_datetime(datos['Planned_Delivery_Start_Date'])
datos['Planned_Delivery_End_Date'] = pd.to_datetime(datos['Planned_Delivery_End_Date'])
#datos['Actual_Delivery_Date'] = pd.to_datetime(datos['Actual_Delivery_Date'])  # Esta fecha es basura

       
datos.rename(columns={'Source ': 'Source'}, inplace=True)  # Le quito el espacio en blanco
datos.loc[datos.Region=='Japan','Territory']='Japan'       # Completo los blancos


In [5]:
# Los datos con geografía desconocida, no aportan (son unos 90 aproximadamente):
datos = datos[~(datos['Territory'].isna()) | ~(datos['Region'].isna())]

# Conversión a USD: 
datos['Total_Amount_Converted'] = (datos['ASP_(converted)']/datos['ASP'])*datos['Total_Amount'] 

# Suma de los montos para cada oportunidad:
datos = datos.groupby('Opportunity_Name').agg({'Total_Amount_Converted':'sum'})\
                .rename(columns={'Total_Amount_Converted':'Total_Amount_Opportunity_converted'})\
                .join(datos.set_index('Opportunity_Name'))\
                .drop(columns='Total_Taxable_Amount')

In [6]:
## Me quedo con los won y lost
datos = datos[(datos['Stage']=='Closed Won') | (datos['Stage']=='Closed Lost')]

# Trasformamos a bool:
#datos['Stage'] = (datos['Stage']=='Closed Won')
datos = datos.drop_duplicates() #por las dudas

#Elige tu propia aventura
Selecciona la region que te interesa analizar:

In [7]:
region = 'APAC' # Japan, EMEA, Americas, APAC, Middle East

# Dale Play para que se arme el dataframe

In [8]:
# En esta sección, además de filtrar la region deseada, armamos nuestro propio dataframe mediante la selección custom de columnas que deseamos
datos_region = datos[datos.Region==region]
datos_region = datos_region[~(datos_region['Territory']=='None')] #limpio los None

# Selección de datos:
datos_unicos_oportunidades = datos_region[['Opportunity_ID','Total_Amount_Opportunity_converted','Territory','Stage','Source','Account_Name','Opportunity_Created_Date','Account_Created_Date',\
                                           'Opportunity_Owner']].drop_duplicates()

In [9]:
# Agrupación de datos

grouped_datos_region = datos_region.groupby('Opportunity_Name').agg({'Opportunity_ID':'count','TRF':'sum'})
grouped_datos_region.columns = ['Items_count','Total_TRF']

opportunities_region = grouped_datos_region.join(datos_unicos_oportunidades).drop_duplicates()
opportunities_region = opportunities_region.rename(columns={'Opportunity_ID':'ID','Total_Amount_Opportunity_converted':'Total_Amount(Converted)'})


# Distintos análisis para la region elegida
El dataframe se llama Opportunities_Region

In [ ]:
# Cantidad de oportunidades distintas para la region:
opportunities_region.index.value_counts().sum()

2059

In [ ]:
# Vistazo al dataframe
opportunities_region.head()

,Items_count,Total_TRF,ID,Total_Amount(Converted),Territory,Stage,Source,Account_Name,Opportunity_Created_Date,Account_Created_Date,Opportunity_Owner
Opportunity_Name,,,,,,,,,,,
Opportunity_Name_0,7,7,1313,3494400.0,NW America,Closed Lost,Source_7,Account_Name_265,2016-03-04,2015-07-30,Person_Name_8
Opportunity_Name_1,7,4,1321,2193100.0,NW America,Closed Lost,Source_7,Account_Name_548,2016-03-05,2013-07-27,Person_Name_33
Opportunity_Name_10023,1,0,316,2870.0,SE America,Closed Won,None,Account_Name_1407,2017-01-03,2016-05-24,Person_Name_38
Opportunity_Name_10041,2,0,7462,101185.0,SW America,Closed Won,Source_7,Account_Name_1437,2018-01-19,2018-01-19,Person_Name_46
Opportunity_Name_1012,2,6,2914,2293200.0,NW America,Closed Won,None,Account_Name_1662,2016-10-29,2013-08-22,Person_Name_8


In [ ]:
# Oportunidades por territorios
opportunities_region.Territory.value_counts()

NW America    874
NE America    518
SE America    332
Central US    145
SW America    102
Canada         61
Carribean      27
Name: Territory, dtype: int64

## A continuación los diferentes análisis hechos para estos datos. Cada Análisis finaliza con un gráfico concluyente mostrando lo observado.

#Analisis 1

Vamos a hacer un analisis estadístico de las oportunidades para cada país de la región.

In [30]:

grouped1 = opportunities_region.groupby(['Territory','Stage'])\
                               .agg({'ID':'count','Total_Amount(Converted)':'sum'})\
                               .rename(columns={'ID':'Opportunities','Total_Amount(Converted)':'Total_Amount'})

grouped1['Mean_Amount'] = grouped1.Total_Amount/grouped1.Opportunities #promedio de monto por oportunidad para ese pais

grouped1.head(10)

Opportunities  Total_Amount   Mean_Amount
Territory   Stage                                                 
Australia   Closed Lost            194  2.335216e+09  1.203720e+07
            Closed Won             383  6.313705e+07  1.648487e+05
Burma       Closed Lost              1  2.842125e+04  2.842125e+04
Cambodia    Closed Lost              2  2.260020e+06  1.130010e+06
            Closed Won               2  1.279005e+05  6.395025e+04
China (PRC) Closed Lost              7  1.233950e+07  1.762786e+06
            Closed Won              45  2.780940e+06  6.179866e+04
India       Closed Lost            368  6.985663e+08  1.898278e+06
            Closed Won             310  8.960563e+07  2.890504e+05
Indonesia   Closed Lost             88  1.175635e+08  1.335949e+06

In [26]:
grouped1.reset_index(inplace=True)
grouped1[grouped1.Stage=='Closed Won'].nlargest(10,'Mean_Amount')

,Territory,Stage,Opportunities,Total_Amount,Mean_Amount
24,Taiwan,Closed Won,7,4.941021e+06,705860.200000
8,India,Closed Won,310,8.960563e+07,289050.415581
26,Thailand,Closed Won,77,1.794616e+07,233067.068701
22,South Korea,Closed Won,6,1.175210e+06,195868.333333
1,Australia,Closed Won,383,6.313705e+07,164848.697046
20,South East Asia,Closed Won,1,8.120000e+04,81200.000000
10,Indonesia,Closed Won,50,4.015942e+06,80318.849000
13,New Zealand,Closed Won,17,1.209215e+06,71130.276471
17,Singapore,Closed Won,216,1.464910e+07,67819.887176
4,Cambodia,Closed Won,2,1.279005e+05,63950.250000


# Analisis 2
Vamos a buscar nuestros mejores clientes de la region

# Analisis 3
Vamos a buscar nuestros mejores vendedores de la region